In [6]:
!pip install sentence_transformers

# Importing necessary libraries
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from tqdm import tqdm
import pandas as pd
import polars as pl
from rich import print as pp
import os
from pandas import DataFrame


In [7]:
path = "../data/sample/"
files = os.listdir(path)

dfs = []

for file_name in files:
    pp(f"[blue bold]> Processing {file_name}...[/blue bold]")
    file_path = os.path.join(path, file_name)
    df_tmp = pl.read_csv(file_path).to_pandas()
    dfs += [df_tmp]

pp(f"[green bold]> Concatenating {len(dfs)} dataframes...[/green bold]")
df: DataFrame = pd.concat(
    dfs,
    ignore_index=True,
)


df.reset_index(drop=True, inplace=True)
df.drop_duplicates(inplace=True)

df.head()

> Processing NYTIMES.csv...

> Processing SEANHANNITY.csv...

> Processing VICE.csv...

> Processing TIME.csv...

> Processing GUARDIAN.csv...

...

> Processing ABC.csv...

> Processing BREITBARTNEWS.csv...

> Concatenating 28 dataframes...

/tmp/ipykernel_3377479/2872775576.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df: DataFrame = pd.concat(


,text,retweet,reply,like,quote,impression,is_reply,id,created_at,author_id,lang,text_raw,media,date,t
0,mom,41.0,6.0,95.0,0.0,0.0,False,7.282756e+17,2016-05-05 17:30:05+00:00,807095.0,en,When your mom takes over your wedding https://...,nytimes,2016-05-05 17:30:05,17
1,donald trump jan brewer,142.0,38.0,307.0,0.0,0.0,False,6.987265e+17,2016-02-14 04:32:40+00:00,807095.0,en,"Before Donald Trump, there was Jan Brewer http...",nytimes,2016-02-14 04:32:40,6
2,american soccer jordan morri ...,14.0,0.0,0.0,0.0,0.0,False,7.703848e+17,2016-08-29 22:17:03+00:00,807095.0,en,RT @NYTSports: The next big thing in American ...,nytimes,2016-08-29 22:17:03,34
3,famili win million powerb jackpot ...,105.0,0.0,0.0,0.0,0.0,False,7.313451e+17,2016-05-14 04:47:03+00:00,807095.0,en,RT @NYTMetro: A family wins a $429 million Pow...,nytimes,2016-05-14 04:47:03,19
4,mormon distast donald trump utah grab...,165.0,37.0,313.0,27.0,0.0,False,7.631597e+17,2016-08-09 23:47:02+00:00,807095.0,en,Mormons’ distaste for Donald Trump puts Utah u...,nytimes,2016-08-09 23:47:02,31


In [26]:
df["text_raw"] = df["text_raw"].astype(str).fillna("")
df["unique_id"] = df.index

In [27]:
df.to_csv("../data/process/sample.agg.csv", index=False)

In [15]:
def grab_text(t=1, df=df):
    # return a list of text for a given t
    return df.loc[df["t"] == t, "text_raw"].tolist()


# grab_text(0)

In [23]:
model = SentenceTransformer("bert-base-nli-mean-tokens")


def cal_sim_score(
    sentences: list = [
        "Three years later, the coffin was still full of Jello.",
        "The fish dreamed of escaping the fishbowl and into the toilet where he saw his friend go.",
        "The person box was packed with jelly many dozens of months later.",
        "He found a leprechaun in his walnut shell.",
    ],
    model: SentenceTransformer = model,
) -> float:
    # Encoding the sentences
    sentence_embeddings = model.encode(sentences)

    # Calculating the cosine similarity between the sentences
    sim_scores = cosine_similarity(sentence_embeddings)
    sim_scores = np.mean(sim_scores) * 100
    sim_scores = round(sim_scores, 3)
    return sim_scores


def cal_sim_score_single(
    sentences: list = [
        "Three years later, the coffin was still full of Jello.",
        "The fish dreamed of escaping the fishbowl and into the toilet where he saw his friend go.",
        "The person box was packed with jelly many dozens of months later.",
        "He found a leprechaun in his walnut shell.",
    ],
    model: SentenceTransformer = model,
) -> list:
    # Filter out non-string sentences to avoid errors
    # sentences = [s for s in sentences if isinstance(s, str)]

    # Encoding the sentences
    sentence_embeddings = model.encode(sentences)

    # Calculating the cosine similarity between the sentences
    sim_scores = cosine_similarity(sentence_embeddings)
    sim_scores = [round(np.mean(score) * 100, 6) for score in sim_scores]
    return list(sim_scores)


# Calculating the similarity scores
# cal_sim_score()
cal_sim_score_single(grab_text(1)[:5])

[53.099751, 55.060023, 51.603186, 45.022029, 48.017597]

In [25]:
ts = sorted(df["t"].unique().tolist())
ts[:3]

[0, 1, 2]

# Run

In [30]:
sim_dfs = []
save_path = "../data/process/sim/"
for t in tqdm(ts, position=0, leave=True):
    pp(f"--- Processing t={t}...")
    df_tmp = df.loc[df["t"] == t]
    ids = df_tmp.loc[df_tmp["t"] == t, "unique_id"].tolist()
    sentences = grab_text(t, df=df_tmp)
    sim_scores = cal_sim_score_single(sentences)
    sim_df = pd.DataFrame(
        {
            "unique_id": ids,
            "t": [t] * len(sentences),
            "text": sentences,
            "similarity_score": sim_scores,
        }
    )
    sim_dfs.append(sim_df)

sim_dfs_agg = pd.concat(sim_dfs)
save_path = f"{save_path}/similarity_scores.csv"
sim_dfs_agg.to_csv(save_path, index=False)

  0%|          | 0/53 [00:00<?, ?it/s]

--- Processing t=0...

  2%|▏         | 1/53 [00:07<06:06,  7.06s/it]

--- Processing t=1...

  4%|▍         | 2/53 [00:15<06:45,  7.95s/it]

...

 96%|█████████▌| 51/53 [07:16<00:16,  8.48s/it]

--- Processing t=51...

 98%|█████████▊| 52/53 [07:23<00:07,  7.98s/it]

--- Processing t=52...

100%|██████████| 53/53 [07:25<00:00,  8.40s/it]
